Subimos archivos desde el dispositivo a la maquina.


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving steam_games_original.csv to steam_games_original.csv
Saving steam_original.csv to steam_original.csv
User uploaded file "steam_games_original.csv" with length 81737147 bytes
User uploaded file "steam_original.csv" with length 5815053 bytes


imports

In [5]:
#import numpy as ny
import pandas as pd

**Merge y refactorizacion de los datasets**

Tenemos dos datasets, "steam_original" que contiene información que nos puede ser útil para el sistema de recomendación, y "steam_games_original" que contiene las descripciones de los juegos que usamos como principal "pivote" de nuestro sistema de recomendación. 
Necesitamos unificarlo todo en un dataset para poder seguir con el sistema de recomendación, además también queremos eliminar columnas que no nos interesan para nuestro sistema, como pueden ser appid, english, negative_ratings, median_playtime, etc.

In [6]:
# Import original .csv
dataSteamDataset = pd.read_csv('steam_original.csv') 
dataSteamGamesDataset= pd.read_csv('steam_games_original.csv')

# pop(name) delete the column 'name' on the .csv
#TODO delete publisher, developer, etc
dataSteamDataset.pop('appid')
dataSteamDataset.pop('english')
dataSteamDataset.pop('negative_ratings')
dataSteamDataset.pop('median_playtime')
dataSteamDataset.pop('owners') 

# Add game description to original dataset, pk= 'name'
dataSteamwithDescription = pd.merge(dataSteamGamesDataset.loc[:, ['name','game_description']],dataSteamDataset, on='name', how='outer') 

print("\nDatasetSteam with Description generated\n") 
dataSteamwithDescription


DatasetSteam with Description generated



,name,game_description,release_date,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,average_playtime,price
0,DOOM,"About This Game Developed by id software, the...",2016-05-12,id Software,Bethesda Softworks,windows,18.0,Single-player;Multi-player;Co-op;Steam Achieve...,Action,FPS;Action;Gore,54.0,68536.0,751.0,14.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,About This Game PLAYERUNKNOWN'S BATTLEGROUND...,2017-12-21,PUBG Corporation,PUBG Corporation,windows,0.0,Multi-player;Online Multi-Player;Stats,Action;Adventure;Massively Multiplayer,Survival;Shooter;Multiplayer,37.0,496184.0,22938.0,26.99
2,BATTLETECH,About This Game From original BATTLETECH/Mec...,2018-04-24,Harebrained Schemes,Paradox Interactive,windows;mac;linux,0.0,Single-player;Multi-player;Online Multi-Player...,Action;Adventure;Strategy,Strategy;Mechs;Turn-Based,128.0,8238.0,1641.0,34.99
3,DayZ,About This Game The post-soviet country of Ch...,2018-12-13,Bohemia Interactive,Bohemia Interactive,windows,18.0,Multi-player;Online Multi-Player;Steam Worksho...,Action;Adventure;Massively Multiplayer,Survival;Zombies;Open World,0.0,118435.0,7235.0,33.99
4,EVE Online,About This Game,2010-12-15,CCP,CCP,windows;mac,0.0,Multi-player;Online Multi-Player;MMO;Co-op;Onl...,Action;Free to Play;Massively Multiplayer;RPG;...,Space;Massively Multiplayer;Sci-fi,0.0,8317.0,5123.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48390,Funny Bunny: Adventures,NaN,2019-04-24,mPower Games Studio,mPower Games Studio,windows;mac;linux,0.0,Single-player;Steam Achievements,Action;Adventure;Casual;Indie,Casual;Adventure;Arcade,22.0,3.0,0.0,7.19
48391,ArchMMO 2,NaN,2019-04-12,CG Creations,CG Creations,windows,0.0,Multi-player;Online Multi-Player;MMO;Online Co-op,Action;Indie;Massively Multiplayer;RPG,Action;RPG;Massively Multiplayer,0.0,3.0,0.0,11.39
48392,The Underground Watcher/地下监察员,NaN,2019-04-22,Yima VLG.,Yima VLG.,windows,0.0,Single-player;Steam Achievements;Partial Contr...,Action;Indie,Action;Indie;Pixel Graphics,25.0,13.0,0.0,10.29
48393,New Yankee 6: In Pharaoh's Court,NaN,2019-04-03,Yustas Game Studio,Alawar Entertainment,windows;mac,0.0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0.0,2.0,0.0,5.19


Eliminamos aquellas tuplas con algún valor nulo

In [8]:
dataSteamwithDescription = dataSteamwithDescription.dropna()

print("\dataSteamwithDescription filtered\n")
dataSteamwithDescription 

\dataSteamwithDescription filtered



,name,game_description,release_date,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,average_playtime,price
0,DOOM,"About This Game Developed by id software, the...",2016-05-12,id Software,Bethesda Softworks,windows,18.0,Single-player;Multi-player;Co-op;Steam Achieve...,Action,FPS;Action;Gore,54.0,68536.0,751.0,14.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,About This Game PLAYERUNKNOWN'S BATTLEGROUND...,2017-12-21,PUBG Corporation,PUBG Corporation,windows,0.0,Multi-player;Online Multi-Player;Stats,Action;Adventure;Massively Multiplayer,Survival;Shooter;Multiplayer,37.0,496184.0,22938.0,26.99
2,BATTLETECH,About This Game From original BATTLETECH/Mec...,2018-04-24,Harebrained Schemes,Paradox Interactive,windows;mac;linux,0.0,Single-player;Multi-player;Online Multi-Player...,Action;Adventure;Strategy,Strategy;Mechs;Turn-Based,128.0,8238.0,1641.0,34.99
3,DayZ,About This Game The post-soviet country of Ch...,2018-12-13,Bohemia Interactive,Bohemia Interactive,windows,18.0,Multi-player;Online Multi-Player;Steam Worksho...,Action;Adventure;Massively Multiplayer,Survival;Zombies;Open World,0.0,118435.0,7235.0,33.99
4,EVE Online,About This Game,2010-12-15,CCP,CCP,windows;mac,0.0,Multi-player;Online Multi-Player;MMO;Co-op;Onl...,Action;Free to Play;Massively Multiplayer;RPG;...,Space;Massively Multiplayer;Sci-fi,0.0,8317.0,5123.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40860,Fruit Tower Defense,About This Game Fruit Tower Defense is a stra...,2018-08-15,Fratboy Entertainment,Fratboy Entertainment,windows;linux,0.0,Single-player;Steam Achievements,Adventure;Casual;Indie;Strategy,Strategy;Adventure;Indie,6.0,5.0,0.0,2.89
40862,ChromaSquares,About This Game ChromaSquares is a puzzle gam...,2018-09-14,Racing Thought Studios,Racing Thought Studios,windows;mac;linux,0.0,Single-player;Steam Achievements,Casual;Indie;Strategy,Strategy;Indie;Casual,20.0,1.0,0.0,3.99
40872,Nanoui,About This Game The book of Nanoui has been ...,2018-03-21,Phoesion,Phoesion,windows,0.0,Single-player;Multi-player;Online Multi-Player...,Violent;Action;Adventure;Casual;Indie;RPG,RPG;Casual;Adventure,0.0,10.0,0.0,5.19
40874,Achievement Collector: Cat,About This Game Achievement Collector: Cat in...,2018-09-06,QUIKIN Games,QUIKIN Games,windows,0.0,Single-player;Steam Achievements;Stats,Action;Adventure,Action;Adventure,100.0,20.0,0.0,20.00


Tokenize

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []
preprocessedTags = []

for row in dataSteamwithDescription.itertuples():
    
    text = word_tokenize(row[3]) ## indice de la columna que contiene el texto, en este caso game_description
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    print(text)
    preprocessedText.append(text)

preprocessedData = dataSteamwithDescription
preprocessedData['processed_text'] = preprocessedText

for row in preprocessedData.itertuples():
    
    
    text = word_tokenize(row[10]) ## indice de la columna que contiene el texto, en este caso tags
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedTags.append(text)

preprocessedData2 = preprocessedData
preprocessedData2['processed_tags'] = preprocessedTags

print('DataSet tokenized')

preprocessedData2

Bag of words

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData2['processed_text'])

textsBoWGD= bagOfWordsModel.transform(preprocessedData['processed_text']) #Game description
textsBoWT= bagOfWordsModel.transform(preprocessedData['processed_tags']) #Tags

print("Bag of Words Finished")

# Calculate the distance TODO crear vector de atributos
distance_matrix= pairwise_distances(textsBoWGD,textsBoWGD ,metric='cosine')

print(distance_matrix.shape)
print(type(distance_matrix))

# Description
searchDescription= " About This Game The Devil you know returns in this brand new entry in the over-the-top action series available on the PC. Prepare to get downright demonic with this signature blend of high-octane stylized action and otherworldly & original characters the series is known for. Director Hideaki Itsuno and the core team have returned to create the most insane, technically advanced and utterly unmissable action experience of this generation! The threat of demonic power has returned to menace the world once again in Devil May Cry 5 . The invasion begins when the seeds of a “demon tree” take root in Red Grave City. As this hellish incursion starts to take over the city, a young demon hunter Nero, arrives with his partner Nico in their “Devil May Cry” motorhome. Finding himself without the use of his right arm, Nero enlists Nico, a self-professed weapons artist, to design a variety of unique mechanical Devil Breaker arms to give him extra powers to take on evil demons such as the blood sucking flying Empusa and giant colossus enemy Goliath. FEATURES High octane stylized action – Featuring three playable characters each with a radically different stylish combat play style as they take on the city overrun with demons Groundbreaking graphics – Developed with Capcom’s in-house proprietary RE engine, the series continues to achieve new heights in fidelity with graphics that utilize photorealistic character designs and stunning lighting and environmental effects. Take down the demonic invasion – Battle against epic bosses in adrenaline fueled fights across the over-run Red Grave City all to the beat of a truly killer soundtrack. Demon hunter – Nero, one of the series main protagonists and a young demon hunter who has the blood of Sparda, heads to Red Grave City to face the hellish onslaught of demons, with weapons craftswoman and new partner-in-crime, Nico. Nero is also joined by stylish, legendary demon hunter, Dante and the mysterious new character, V. "

indexOfDescription = preprocessedData[preprocessedData['game_description']==searchDescription].index.values[0]

distance_scores = list(enumerate(distance_matrix[indexOfDescription]))

ordered_scores = sorted(distance_scores, key=lambda x: x[1])

top_scores = ordered_scores[1:11]
top_indexes = [i[0] for i in top_scores]



print(preprocessedData['name'].iloc[top_indexes])


ValueError: ignored